In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
# import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import numpy as np
import PIL 

In [2]:
import os
import sys

MODEL_PATH = '../models'
if MODEL_PATH not in sys.path:
    sys.path.append(MODEL_PATH)

in_dir='../dataset/color/'
img_paths = [x.path for x in os.scandir(in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
len(img_paths)

5578

In [3]:
loader = torchvision.transforms.Compose([
#     transforms.Scale(imsize), 
    torchvision.transforms.ToTensor()
])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = PIL.Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image #assumes that you're using GPU

image = image_loader(img_paths[0])

In [4]:
import resnet_detector

In [5]:
model = resnet_detector.DetectorModel()

In [6]:
optimizer = optim.SGD(model.parameters(),lr=0.001)

optimizer.zero_grad()
score_maps_list,endpoints = model(image)

1
1
1


## 从网络输出得到patch

In [7]:
score_maps_list[7].shape

torch.Size([1, 1, 157, 210])

# 为了得到需要的多尺度特征点，需要在上面基础网络上再封装

## build_multi_scale_deep_detector_3DNMS

In [8]:
photos=image
batch_size = photos.shape[0]
height = photos.shape[2]
width = photos.shape[3]
C = photos.shape[1]

In [11]:
scale_factors = endpoints['scale_factors']
scale_factors_tensor=torch.tensor(scale_factors)
num_scale = len(score_maps_list)
scale_logits = [None] * num_scale

```
def instance_normalization(inputs):
    # normalize 0-means 1-variance in each sample (not take batch-axis)
    inputs_dim = inputs.get_shape().ndims
    # Epsilon to be used in the tf.nn.batch_normalization
    var_eps = 1e-3    
    if inputs_dim == 4:
        moments_dims = [1,2] # NHWC format
    elif inputs_dim == 2:
        moments_dims = [1]
    else:
        raise ValueError('instance_normalization suppose input dim is 4: inputs_dim={}\n'.format(inputs_dim))
    mean, variance = tf.nn.moments(inputs, axes=moments_dims, keep_dims=True)
    outputs = tf.nn.batch_normalization(inputs, mean, variance, None, None, var_eps) # non-parametric normalization
    return outputs
```
================torch.nn.BatchNorm2d(1, eps=1e-06, momentum=None, affine=None, track_running_stats=None)

In [44]:
instance_normalization=torch.nn.BatchNorm2d(1, eps=1e-06, momentum=None, affine=None, track_running_stats=None)

### spatial_softmax

In [ ]:
for i in range(num_scale):
    logits = instance_normalization(score_maps_list[i])
    _heatmaps = spatial_softmax(logits, config.sm_ksize, config.com_strength)
    _heatmaps = tf.image.resize_images(_heatmaps, (height, width)) # back to original resolution
    multi_scale_heatmaps[i] = _heatmaps

In [36]:
x=torch.arange(0,9.0,1).reshape([1,1,3,3])
x

tensor([[[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]]]])

In [45]:
ba(x)

tensor([[[[-1.5492, -1.1619, -0.7746],
          [-0.3873,  0.0000,  0.3873],
          [ 0.7746,  1.1619,  1.5492]]]])

In [22]:
torch.var(x.view(1, -1, x.shape[2], x.shape[3]), dim=3, keepdim=True,unbiased=False)

tensor([[[[0.1248],
          [0.0620],
          [0.1807],
          [0.1100],
          [0.0572],
          [0.0568],
          [0.0593],
          [0.0564],
          [0.0508],
          [0.0610],
          [0.0539],
          [0.0659],
          [0.0690],
          [0.0579],
          [0.0638],
          [0.0942],
          [0.0777],
          [0.0793],
          [0.0909],
          [0.0740],
          [0.0856],
          [0.0851],
          [0.0954],
          [0.0862],
          [0.0875],
          [0.0909],
          [0.1084],
          [0.1036],
          [0.0929],
          [0.0936],
          [0.0904],
          [0.0952],
          [0.0903],
          [0.1181],
          [0.0818],
          [0.0680],
          [0.0813],
          [0.0865],
          [0.0682],
          [0.0800],
          [0.0824],
          [0.0749],
          [0.0912],
          [0.0908],
          [0.0919],
          [0.0908],
          [0.0874],
          [0.0719],
          [0.0751],
          [0.0927],


In [58]:
m = nn.BatchNorm2d(1, momentum=None, affine=False)

In [59]:
m(x)

tensor([[[[-1.1682, -2.2953, -2.8026,  ..., -4.7189, -1.0219,  0.9656],
          [-0.1638, -1.3107, -2.3589,  ..., -3.2995, -1.6894, -1.0583],
          [ 2.0597,  0.0909, -1.4558,  ..., -6.4776, -3.1175,  0.0154],
          ...,
          [-3.1242, -1.5743,  1.5801,  ...,  2.6838,  2.7484,  1.7357],
          [-0.6293,  2.4569, -0.7084,  ...,  1.9609,  2.7929,  1.0595],
          [-1.8180,  2.7054,  0.7647,  ...,  4.1476,  2.6694,  0.8345]]]],
       grad_fn=<NativeBatchNormBackward>)

In [ ]:
def build_patch_extraction(config, det_endpoints, photos=None, name='PatchExtract'):
    with tf.name_scope(name):
        batch_inds = det_endpoints['batch_inds']
        kpts = det_endpoints['kpts']
        kpts_scale = det_endpoints['kpts_scale']
        kpts_ori = det_endpoints['kpts_ori']

        if config.desc_inputs == 'det_feats':
            feat_maps = tf.identity(det_endpoints['feat_maps']) 
        elif config.desc_inputs == 'photos':
            feat_maps = tf.identity(photos)
        elif config.desc_inputs == 'concat':
            feat_maps = tf.concat([photos, det_endpoints['feat_maps']], axis=-1)
        else:
            raise ValueError('Unknown desc_inputs: {}'.format(config.desc_inputs))

        patches = transformer_crop(feat_maps, config.patch_size, batch_inds, kpts,
                        kpts_scale=kpts_scale, kpts_ori=kpts_ori)

        return patches

In [ ]:
def extract_keypoints(top_k):
    coords = tf.where(tf.greater(top_k, 0.))
    num_kpts = tf.reduce_sum(top_k, axis=[1,2,3])
    coords = tf.cast(coords, tf.int32)
    batch_inds, kp_y, kp_x, _ = tf.split(coords, 4, axis=-1)
    batch_inds = tf.reshape(batch_inds, [-1])
    kpts = tf.concat([kp_x, kp_y], axis=1)

    num_kpts = tf.cast(num_kpts, tf.int32)
    # kpts: [N,2] (N=B*K)
    # batch_inds: N,
    # num_kpts: B
    return kpts, batch_inds, num_kpts

In [10]:
def transformer_crop(
    images, out_size, 
    batch_inds, 
    kpts_xy, 
    kpts_scale=None, 
    kpts_ori=None, 
    thetas=None):
    # images : [B,C,H,W]
    # out_size : (out_width, out_height)
    # batch_inds : [B*K,] tf.int32 [0,B)
    # kpts_xy : [B*K,2] tf.float32 or whatever
    # kpts_scale : [B*K,] tf.float32
    # kpts_ori : [B*K,2] tf.float32 (cos,sin)
    if isinstance(out_size, int):
        out_width = out_height = out_size
    else:
        out_width, out_height = out_size
    hoW = out_width // 2
    hoH = out_height // 2

    num_batch = images.shape[0]
    height = images.shape[2]
    width = images.shape[3]
    C = images.shape[1]
    
    
    

In [27]:
min_scale=2**-3
max_scale=1
num_scales=5

scale_log_factors = np.linspace(np.log(max_scale), np.log(min_scale), num_scales)
scale_factors = np.exp(scale_log_factors)
scale_factors

array([1.        , 0.59460356, 0.35355339, 0.2102241 , 0.125     ])

In [12]:
import gc
gc.collect()

20